In [2]:
import json
import pandas as pd
import gc
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [3]:
def load_meta_data():
    gc.collect()
    with open(f'../data/amazon_meta.json', 'r') as json_file :
        for line in json_file :
            data = json.loads(line)
            yield data['asin'], data['category'], data['description'], data['title'], data['brand'], data['feature']

In [4]:
arr = []
for data in tqdm(load_meta_data()) :
    arr.append(data)

2934949it [01:11, 40942.98it/s]


In [5]:
movies_df = pd.DataFrame(arr, columns=['asin', 'category', 'description', 'title', 'brand', 'feature'])

In [6]:
movies_df.drop((movies_df[movies_df['asin'].str.match('B')]).index, inplace=True)

In [7]:
# First let's make a copy of the movies_df
movies_with_genres = movies_df.copy(deep=True)

# Let's iterate through movies_df, then append the movie genres as columns of 1s or 0s.
# 1 if that column contains movies in the genre at the present index and 0 if not.

x = []
for index, row in movies_df.iterrows():
    x.append(index)
    for genre in row['category']:
        movies_with_genres.at[index, genre] = 1

# Confirm that every row has been iterated and acted upon
print(len(x) == len(movies_df))

movies_with_genres.head(3)


True


,asin,category,description,title,brand,feature,Books,"New, Used & Rental Textbooks",Medicine & Health Sciences,Arts & Photography,...,Beijing,Barcelona,Turkey,Istanbul,Portugal,Lisbon,Social Services & Welfare,Children's & Teens,Customs &amp; Traditions,Ecclesiology
0,0000092878,[],[It is a biology book with God&apos;s perspect...,Biology Gods Living Creation Third Edition 10 ...,Keith Graham,[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000047715X,"[Books, New, Used & Rental Textbooks, Medicine...",[],Mksap 16 Audio Companion: Medical Knowledge Se...,Acp,[],1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000004545,"[Books, Arts & Photography, Music]","[Discography of American Punk, Hardcore, and P...","Flex! Discography of North American Punk, Hard...",Burkhard Jarisch,[],1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def load_rating_data():
    gc.collect()
    with open(f'../data/amazon_rating.json', 'r') as json_file :
        for line in json_file :
            data = json.loads(line)
            yield data['reviewerID'], data['asin'], data['overall']

In [9]:
arr = []
for data in tqdm(load_rating_data()):
    arr.append(data)

51311621it [09:15, 92352.95it/s] 


In [10]:
ratings = pd.DataFrame(arr, columns=['user', 'item', 'rating'])

In [11]:
ratings

,user,item,rating
0,A1C6M8LCIX4M6M,0001713353,5.0
1,A1REUF3A1YCPHM,0001713353,5.0
2,A1YRBRK2XM5D5,0001713353,5.0
3,A1V8ZR5P78P4ZU,0001713353,5.0
4,A2ZB06582NXCIV,0001713353,5.0
...,...,...,...
51311616,A3P8PUZFHWFK1E,B01HJDS76Y,4.0
51311617,A26Q9T9G9MTX9S,B01HJDS76Y,1.0
51311618,AB9M1MQPBAS2J,B01HJEB422,5.0
51311619,A2TO93KMH3DJIK,B01HJEB422,5.0


In [11]:
ratings.drop((ratings[ratings['item'].str.match('B')]).index, inplace=True)

In [14]:
user_rating = ratings[ratings['user'] == 'A1REUF3A1YCPHM']

In [15]:
user_rating['item']

1           0001713353
7701341     0330293133
8201568     0345522478
8342065     034554854X
8957911     0375432329
9307764     0385338139
9315425     0385338163
11533097    0451225546
13275281    0590417010
13466899    061350125X
13719812    0615752314
14032637    0671319418
14388311    0692268693
16244447    0765301482
19337049    0979573874
23285203    1441706321
24366425    1461084741
25902161    1479165077
26254703    1481952439
27014966    1492974900
32538560    1612185592
34037960    1925342891
34882727    1941987834
37045389    B0070Z65FC
43145388    0979573831
43202646    0982202997
43226885    0982750587
43324279    0985147105
43411433    0988904942
43609711    099827920X
44276422    143570942X
44375627    1442123745
44488709    1451638027
44630234    1461189276
44857473    1478103590
44873887    1478353287
45107264    1492106062
45154970    1493534378
45496045    1511535881
45590466    1517221951
45951610    1546768351
46773078    1595070699
46775619    1595071407
46776298   

In [16]:
user_genre_df = pd.DataFrame(columns=movies_with_genres.columns)

In [17]:
for i in range(len(movies_with_genres)):
    if movies_with_genres.loc[i,'asin'] in list(user_rating['item']):
        user_genre_df = user_genre_df.append(movies_with_genres.iloc[i,:])

In [18]:
user_genre_df.reset_index(drop=True, inplace=True)

In [19]:
user_genre_df.drop(['asin', 'category', 'description', 'title', 'brand', 'feature', 'Books'], axis=1, inplace=True)

In [20]:
user_genre_df=user_genre_df.fillna(0.0)

In [21]:
print(user_rating.shape)
print(user_genre_df.T.shape)

(50, 3)
(1072, 49)


In [26]:
user_rating.rating.reset_index(drop=True, inplace=True)

In [27]:
user_profile = user_genre_df.T.dot(user_rating.rating)

ValueError: matrices are not aligned

In [24]:
user_profile.sort_values(ascending=False)[:10]

NameError: name 'user_profile' is not defined

In [ ]:
movies_with_genres = movies_with_genres.set_index(movies_with_genres.asin)
movies_with_genres.head()

: 

: 

In [ ]:
movies_with_genres

NameError: name 'movies_with_genres' is not defined